In [ ]:
'''
1)  Alanine (Ala, A)
2)  Arginine (Arg, R)
3)  Asparagine (Asn, N)
4)  Aspartic acid (Asp, D)
5)  Cysteine (Cys, C)
6)  Glutamic acid (Glu, E)
7)  Glutamine (Gln, Q)
8)  Glycine (Gly, G)
9)  Histidine (His, H)
10) Isoleucine (Ile, I)
11) Leucine (Leu, L)
12) Lysine (Lys, K)
13) Methionine (Met, M)
14) Phenylalanine (Phe, F)
15) Proline (Pro, P)
16) Serine (Ser, S)
17) Threonine (Thr, T)
18) Tryptophan (Trp, W)
19) Tyrosine (Tyr, Y)
20) Valine (Val, V)
'''
amino_acid = {"A":1, "R":2, "N":3, "D":4, "C":5, 
              "E":6, "Q":7, "G":8, "H":9, "I":10, 
              "L":11, "K":12, "M":13, "F":14, "P":15, 
              "S":16, "T":17, "W":18, "Y":19, "V":20}
def AA_to_token(x, dictionary=amino_acid):
    if AA in dictionary.keys():
        return dictionary[AA]
    else:
        return 0
def AA_to_onehot(x, dictionary=amino_acid):
    keys = dictionary.keys()
    m = max(dictionary.values())
    if x in keys:
        token = dictionary[x]
    else:
        token = 0
    eyes = np.eye(m+1, m, k=-1)
    return eyes[token]

    
secondary_structure = {"C":0  ,"H":1, "E":2} 
'''
0) C: not structured
1) H: alpha-helix
2) E: beta-sheet(strand)
'''
def SS_to_token(SS):
    if SS in secondary_structure.keys():
        return secondary_structure[SS]
    else:
        return 0
def SS_to_onehot(x, dictionary=secondary_structure):
    keys = dictionary.keys()
    m = max(dictionary.values())
    token = dictionary[x]
    eyes = np.eye(m+1, m, k=-1)
    return eyes[token]

side_chain = {"None":0,
              "G":1, "gly":1,
              "V":2, "A":2, "very_small":2,
              "S":3, "I":3, "L":3, "T":3, "C":3, "small":3,
              "D":4, "E":4, "N":4, "Q":4, "M":4, "normal":4,
              "R":5, "K":5, "long":5,
              "F":6, "W":6, "Y":6, "H":6, "cycle":6,
              "P":7, "pro":7}
def SC_to_token(SC):
    if SC in side_chain.keys():
        return side_chain[SC]
    else:
        return 0
def SC_to_onehot(x, dictionary=side_chain):
    keys = dictionary.keys()
    m = max(dictionary.values())
    token = dictionary[x]
    eyes = np.eye(m+1, m, k=-1)
    return eyes[token]
    
phipsi_angle = {"other":0, "beta":1, "alpha": 2}
def Angle_to_token(Angle):
    if Angle in phipsi_angle.keys():
        return phipsi_angle[Angle]
    else:
        return 0
def Angle_to_onehot(x, dictionary=phipsi_angle):
    keys = dictionary.keys()
    m = max(dictionary.values())
    token = dictionary[x]
    eyes = np.eye(m+1, m, k=-1)
    return eyes[token]
    
def OnehotEncoder(series, feature_type):
    if feature_type == 'SEQ':
        token = series.apply(AA_to_token).to_numpy()
        eyes = np.eye(len(amino_acid,keys()+1), k=-1)
        
        
    

In [ ]:
### generate GNN dataset###

win_size = 10
win_len = 2*win_size+1

for name in protein_list:
    name_list = []
    adj_list  = []
    feat_list = []
    
    data = dataset_onehot[dataset_onehot['protein']==name]
    ST_index = np.where((data['SEQ_S']==1) | (data['SEQ_T']==1))[0]
    for index in ST_index:
        adj_matrix = np.eye(win_len,win_len, 1) + np.eye(win_len,win_len) + np.eye(win_len,win_len, -1)
        if index - win_size < 0:
            zeros = np.zeros((win_size-index, win_len))
            adj = adj_matrix[win_size-index:]
            adj_matrix = np.concatenate([zeros, adj], axis=0)

        elif index + win_size >= len(dataset):
            adj = adj_matrix[:-(index+win_size-len(dataset)+1)]
            zeros = np.zeros((index+win_size-len(dataset)+1, win_len))
            adj_matrix = np.concatenate([adj, zeros], axis=0)

        else:
            feat_matrix = dataset.iloc[index-win_size:index+win_size+1]

        name_list.append(name)
        adj_list.append(adj_matrix)
        feat_list.append(feat_matrix)